# Entendre Rank – Colab Notebook

In [ ]:
!pip -q install spotipy lyricsgenius pandas numpy scikit-learn faiss-cpu tqdm transformers torch umap-learn python-dotenv

In [ ]:
import os
# Set secrets here or via Colab's UI
os.environ['SPOTIPY_CLIENT_ID'] = os.getenv('SPOTIPY_CLIENT_ID', 'YOUR_CLIENT_ID')
os.environ['SPOTIPY_CLIENT_SECRET'] = os.getenv('SPOTIPY_CLIENT_SECRET', 'YOUR_CLIENT_SECRET')
os.environ['SPOTIPY_REDIRECT_URI'] = os.getenv('SPOTIPY_REDIRECT_URI', 'http://localhost:8888/callback')
os.environ['GENIUS_ACCESS_TOKEN'] = os.getenv('GENIUS_ACCESS_TOKEN', '')
PLAYLIST_URL = "https://open.spotify.com/playlist/7MNBsBwgsqAsRZkdNE4E5Y"


In [ ]:
import pandas as pd, numpy as np
from src.ingest_spotify import get_spotify_client, fetch_playlist_tracks, fetch_audio_features
from src.ingest_genius import fetch_lyrics_frame
from src.embed_lyrics import SBertEmbedder
from src.build_index import build_vectors, build_faiss_index
from src.recommend import recommend
from src.config import SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI, GENIUS_ACCESS_TOKEN

sp = get_spotify_client(SPOTIPY_CLIENT_ID, SPOTIPY_CLIENT_SECRET, SPOTIPY_REDIRECT_URI)
df_tracks = fetch_playlist_tracks(sp, PLAYLIST_URL)
df_feats  = fetch_audio_features(sp, df_tracks['track_id'].tolist())
df = df_tracks.merge(df_feats, on='track_id', how='left')
df.head()

In [ ]:
# Optional: lyrics + embeddings
use_lyrics = True if os.environ.get('GENIUS_ACCESS_TOKEN') else False
if use_lyrics:
    df = fetch_lyrics_frame(df, os.environ['GENIUS_ACCESS_TOKEN'])
    texts = [t if isinstance(t,str) and t else '' for t in df['lyrics'].tolist()]
    embedder = SBertEmbedder()
    lyr_emb = embedder.encode(texts)
else:
    lyr_emb = None


In [ ]:
# Build vectors + FAISS and test a recommendation
import faiss, numpy as np
vec, scaler = build_vectors(df, lyrics_emb=lyr_emb, alpha=0.6)
index = build_faiss_index(vec)
print('Index size:', index.ntotal)

seed_id = df.iloc[0]['track_id']
recs = recommend(df, vec, index, [seed_id], k=30)
recs.head(10)

In [ ]:
# Save artifacts
import joblib, os, numpy as np, faiss
os.makedirs('artifacts', exist_ok=True)
df.to_parquet('artifacts/track_meta.parquet', index=False)
np.save('artifacts/vectors.npy', vec)
joblib.dump(scaler, 'artifacts/scaler.pkl')
faiss.write_index(index, 'artifacts/faiss.index')
print('Saved artifacts to ./artifacts')